# CryptOR

<b><i style="font-size:13px">Tags: </i></b><i style="font-size:11px">Funciones a trozos</i>

## Enunciado
CryptOR is una compañía que se dedica a minar criptomonedas. El mayor problema de planeación para CryptOR es la compra de energía. Dado el alto consumo de energía, CryptOR debe negociar contratos para los próximos 12 meses con las diferentes empresas de generación eléctrica. El costo por kilovatio-hora (kWh) varía según la cantidad de energía contratada y la empresa generadora. El costo total en función del consumo está representado por funciones no lineales suministradas por cada generadora. Así mismo, cada empresa generadora establece la duración de cada contrato que se firme con la misma y el consumo máximo mensual que pueden suministrar. Los contratos de cada generadora se pueden renovar múltiples veces durante los 12 meses de planeación, pero estos no se deben traslapar. Finalmente, se conoce un pronóstico de la demanda de energía en kWh que se debe satisfacer sin holgura. Las Tablas 1 y 2 resumen toda la información disponible. 

### Tabla 1. Términos de los contratos para cada generadora.

|                     |**Función de costo**                      | **Duración del contrato**| **Consumo máximo mensual**|
|:-------------------:|:----------------------------------------:|:------------------------:|:-------------------------:|
|**Generadora Carbón**|$f_1(x) = 0.0012 x^2$                     | 2 meses                  | 250 kWh                   |
|**Generadora Solar** |$f_2(x) = \begin{cases}0.0028 x^2 - 0.5x + 35 \ \ x>0 \\ 0 \ \ x=0 \end{cases}$          | 4 meses                  | 250 kWh                   |
|**Generadora Eólica**|$f_3(x) = \begin{cases}0.5 x log(\frac{x}{150}) + 30 \ \ x>0 \\ 0 \ \ x=0 \end{cases}$  | 3 meses                  | 250 kWh                   |

### Tabla 2. Demanda mensual en kWh.
|Mes| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|demanda| 100| 150| 300| 300| 300| 500| 400| 400| 200| 200| 200| 200|

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def f1(x):
    # Función de costo de la generadora carbón
    return 0.0012 * (x ** 2)
def f2(x):
    # Función de costo de la generadora solar
    return 0.0028 * x ** 2 - 0.5*x + 35
def f3(x):
    # Función de costo de la generadora eólica
    return 0.5 * x*np.log(x/150) + 30

# Puntos de evaluación de la función
kwh = np.arange(1,250)

fig, axs = plt.subplots(1,3, sharey=False, sharex=True, figsize=(15,5))
axs[0].plot(kwh, f1(kwh))
axs[1].plot(kwh, f2(kwh))
axs[2].plot(kwh, f3(kwh))
for ax_ix, ax in enumerate(axs):
    ax.set_xlabel("Consumo en kWh")
    ax.set_ylabel(f"f {ax_ix+1} (x)")
    ax.set_ylim([0,100])



## Linealización a trozos

Optimizar utilizando las funciones $f_1(x)$, $f_2(x)$ y $f_3(x)$ es computacionalmente más difícil debido a que son funciones no lineales. Sin embrago, dado que son funciones [convexas](https://es.wikipedia.org/wiki/Funci%C3%B3n_convexa) es posible aproximarlas con múltiples funciones lineales. Dada una función convexa $f(x)$, la función $g(x;x_0) = f(x_0) + f'(x_0) * (x-x0)$ es una aproximación de [primer orden](https://es.wikipedia.org/wiki/Serie_de_Taylor), la cual es exacta en $x_0$ y se degrada a medida que $x$ se aleja de $x_0$. Con esta idea en mente, cualquier función convexa se puede aproximar generando múltiples funciones de forma  $g(x;x_0)$ para diferentes valores de $x_0$. La siguiente celda muestra la función $f(x) = (x - 4)^2$ (en azul) y su aproximación (en rojo) utilizando 4 rectas correspondientes a  $g(x;x_0)$ para $x_0=0,3,5,8$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def f(x, orden=0):
    if orden == 0:
        # Función (x-4) ^ 2
        return  (x-4) ** 2 
    else:
        # Derivada evaluada en x
        return 2 *(x - 4)

# Puntos de evaluacion de la función
dominio = np.arange(-2,10,0.01)

fig, axs = plt.subplots(1,1, sharey=False, sharex=True, figsize=(15,5))
axs.plot(dominio, f(dominio))
for x_0 in [0, 3, 5, 8]:
    axs.plot(dominio, f(x_0) +  f(x_0,1) * (dominio - x_0), linestyle='dashed', color='red')
    print(f'Función evaluada en {x_0}:  {f(x_0):.1f} + {f(x_0, 1):.1f} * (x - {x_0})')
axs.set_xlabel("Consumo en kWh")
axs.set_ylabel(f"f(x)")
axs.set_ylim([-2,40])


### Optimizando con la aproximación
Para una función convexa, $f(x)$, los siguientes problemas son equivalentes:
 $$
\begin{align*}
\min & f(x)      &= \ \ \min \ \ &z  \\ 
&x\in \mathbb{R}. &        & z \geq f(x), \\
&                &        & x \in \mathbb{R}.
\end{align*}
$$
Asi mismo, $f(x)\geq g(x,x_0)$ para cualquier valor de $x_0$. 

Con estas dos observaciones, el problema original, $\min_{x\in\mathbb{R}} (x-4)^2$, lo podemos aproximar con el siguiente modelo:
 $$
\begin{align*}
\min \ \ &z  \\ 
       & z \geq 16  -8 * (x - 0) \\
       & z \geq 1  -2 * (x - 3) \\
       & z \geq 1 + 2 * (x - 5) \\
       & z \geq 16 + 8 * (x - 8) \\
      & x \in \mathbb{R} 
\end{align*}
$$


### Funciones de costo aproximadas
Volviendo al problema de cryptOR, una posible aproximación se muestra a continuación. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def f1(x, orden=0):
    if orden == 0:
        # Función de costo de la generadora carbón
        return 0.0012 * (x ** 2) 
    else:
        # Derivada evaluada en x
        return 0.0012 * 2 * x

def f2(x, orden=0):
    if orden == 0:
        # Función de costo de la generadora solar
        return 0.0028 * x ** 2 - 0.5*x + 35
    else:
        # Derivada evaluada en x
        return 2 * 0.0028 * x - 0.5
def f3(x, orden=0):
    if orden == 0:
        # Función de costo de la generadora eólica
        return 0.5 * x*np.log(x/150) + 30
    else:
        # Derivada evaluada en x
        return 0.5 * np.log(x/150) + 0.5

# Funciones de costo
f = [f1,f2,f3]
# Puntos de evaluación de la función
kwh = np.arange(1,250)

fig, axs = plt.subplots(1,3, sharey=False, sharex=True, figsize=(15,5))
for i, ax in enumerate(axs):
    axs[i].plot(kwh, f[i](kwh), )
    for x_0 in [20, 50, 100, 200]:
        axs[i].plot(kwh, f[i](x_0) +  f[i](x_0,1) * (kwh - x_0), linestyle='dashed', color='red')
        print(f'Función {i+1} evaluada en {x_0}:  {f[i](x_0):.3f} + {f[i](x_0, 1):.3f} * (x - {x_0})')
    ax.set_xlabel("Consumo en kWh")
    ax.set_ylabel(f"f {i+1} (x)")
    ax.set_ylim([-1,100])


<div style="text-align:justify">
Formule un modelo general de optimización lineal que le permita a CryptOR satisfacer la demanda de energía al mínimo costo y satisfaciendo las condiciones de los contratos. Para esto usted debe seguir los siguientes pasos: 

## Formulación

**1.** Escriba lo(s) conjunto(s), parámetro(s) y variable(s) de decisión que utilizará en el modelo.  

### Conjuntos
- $P$: Generadoras
- $T$: Periodos de planeación

### Parámetros
- $d_t$: demanda de energía en kWh para el mes $t\in T$
- $b_i$: Consumo máximo mensual de la generadora $i\in G$
- $m_i$: Duración de cada contrato de la generadora $i\in G$
- $a_{ik}$: intercepto de la ecuación $k\in{1,2,3,4}$ para la generadora $i\in G$
- $p_{ik}$: pendiente de la ecuación $k\in{1,2,3,4}$ para la generadora $i\in G$

### Variables de decisión
- $x_{it}$: cantidad de kWh contratados a la generadora $i\in G$ para el mes $t\in T$
- $y_{it}$: variable binaria que indica si un nuevo contrato se firma con la generadora $i\in G$ en el mes $t\in T$
- $v_{it}$: variable binaria que indica si un contrato con la generadora $i\in G$ esta vigente en el mes $t\in T$
- $z_{it}$: costo total por consumo de energía de la generadora $i\in G$ en el mes $t\in T$

### Restricciones
**2.** Escriba la(s) restricción(es) lineal(es) que garantiza(n) que se debe respetar la demanda exacta en cada mes. 


$$
\begin{align*}
\sum_{i \in G}x_{it} = d_t \ \ \forall t \in T.
\end{align*}
$$

**3.** Escriba la(s) restricción(es) lineal(es) que garantiza(n) que la cantidad de kWh contratados no superan el consumo máximo mensual en los meses en donde hay un contrato vigente.
$$
\begin{align*}
x_{it} \leq b_i v_{it} \forall i \in G, t \in T.
\end{align*}
$$

**4.** Escriba la(s) restricción(es) lineal(es) que garantiza(n) que si un contrato se firma, debe estar vigente por la duración estipulada, y que solo se puede haber un contrato vigente si se firmó previamente o en el mismo mes.
$$
\begin{align*}
\sum_{t' \in \{\max\{t-m_i+1, 1\}, \ldots, t\}}y_{it'} \geq  v_{it} \ \ \forall i \in G, t \in T. \\ 
|\{t, \ldots, \min\{t+m_i-1,12\}\}|y_{it} \leq  \sum_{t' \in \{t, \ldots, \min\{t+m_i-1,12\}\}}v_{it} \ \ \forall i \in G, t \in T. \\ 
\end{align*}
$$

**4.** Escriba la(s) restricción(es) lineal(es) que garantiza(n) que los contratos con cada generadora no se deben traslapar.
$$
\begin{align*}
\sum_{t' \in \{t, \ldots, t+m_i-1\}}y_{it'} \leq 1 \ \ \forall i \in G, t \in T.
\end{align*}
$$

**4.** Escriba la(s) restricción(es) lineal(es) que modelan las funciones de costo con expresiones lineales. Note que si no hay un contrato vigente en un mes particular, el costo es cero.
$$
\begin{align*}
z_{it} \geq a_{ik} + p_{ik} x_{it} - M(1-v_{it})  \ \ \forall i \in G, t \in T, k \in \{1,2,3,4\}
\end{align*}
$$

**4.** Escriba la(s) restricción(es) que describe(n) matemáticamente el tipo de variable(s) que está utilizando dentro del modelo. 

$$
\begin{align*}
x_{it} & \ge 0 \ \ &&\forall i\in G,t\in T.\\
y_{it} & \in\{0,1\} \ \ &&\forall i\in G,t\in T.\\
v_{it} & \in \{0,1\} \ \  &&\forall i\in G,t\in T.\\
z_{it} & \ge 0 \ \ &&\forall i\in G,t\in D.\\
\end{align*}
$$

### Función Objetivo
**5.** Escriba la función objetivo.

$$
\text{minimizar }  \sum_{i\in P}\sum_{t\in T}z_{it}
$$

## Formulación matemática completa

## Formulación

**1.** Escriba lo(s) conjunto(s), parámetro(s) y variable(s) de decisión que utilizará en el modelo.  

### Conjuntos
- $P$: Generadoras
- $T$: Periodos de planeación

### Parámetros
- $d_t$: demanda de energía en kWh para el mes $t\in T$
- $b_i$: Consumo máximo mensual de la generadora $i\in G$
- $m_i$: Duración de cada contrato de la generadora $i\in G$
- $a_{ik}$: intercepto de la ecuación $k\in{1,2,3,4}$ para la generadora $i\in G$
- $p_{ik}$: pendiente de la ecuación $k\in{1,2,3,4}$ para la generadora $i\in G$

### Variables de decisión
- $x_{it}$: cantidad de kWh contratados a la generadora $i\in G$ para el mes $t\in T$
- $y_{it}$: variable binaria que indica si un nuevo contrato se firma con la generadora $i\in G$ en el mes $t\in T$
- $v_{it}$: variable binaria que indica si un contrato con la generadora $i\in G$ está vigente en el mes $t\in T$
- $z_{it}$: costo total por consumo de energía de la generadora $i\in G$ en el mes $t\in T$

**Modelo:**

$$
\begin{align*}

&\text{minimizar } & \sum_{i\in P}\sum_{t\in T}z_{it} &&(1)\\
&\text{sujeto a}&&& \\
&&\sum_{i \in G}x_{it} = d_t \ \  \ \  & \forall t \in T; &(2) \\
&&x_{it} \leq b_i v_{it}  \ \  \ \  & \forall i \in G, t \in T;&(3)\\
&&\sum_{t' \in \{\max\{t-m_i+1, 1\}, \ldots, t\}}y_{it'} \geq  v_{it}  \ \  \ \  & \forall i \in G, t \in T; &(4)\\ 
&&|\{t, \ldots, \min\{t+m_i-1,12\}\}|y_{it} \leq  \sum_{t' \in \{t, \ldots, \min\{t+m_i-1,12\}\}}v_{it}  \ \  \ \  & \forall i \in G, t \in T;&(5)\\
&&\sum_{t' \in \{t, \ldots, t+m_i-1\}}y_{it'} \leq 1  \ \  \ \  & \forall i \in G, t \in T;&(6) \\ 
&&z_{it} \geq a_{ik} + p_{ik} x_{it} - M(1-v_{it})   \ \  \ \  & \forall i \in G, t \in T, k \in \{1,2,3,4\};&(7) \\
&&x_{it}  \ge 0  \ \  \ \  &\forall i\in G,t\in T; &(8)\\
&&y_{it}  \in\{0,1\}  \ \  \ \  &\forall i\in G,t\in T;\\
&&v_{it}  \in \{0,1\}   \ \  \ \  &\forall i\in G,t\in T;\\
&&z_{it}  \ge 0  \ \  \ \  &\forall i\in G,t\in D.
\end{align*}
$$

## Implementación
**6.** Resuelva el modelo planteado utilizando la librería de PulP en Python. ¿Cuál es la solución
óptima del problema? 

In [ ]:
#Importar librerías
import pulp as lp
import numpy as np
from itertools import product

#-----------------
# Conjuntos
#-----------------
# Generadoras
G=["Generadora 1","Generadora 2","Generadora 3",]

# Periodos de planeación
T = list(range(1,12+1)) # range excluye el final del intervalo.

# Conjunto de tuplas (i,t) donde i es una generadora
# en G y t es un periodo de tiempo en T. 
G_x_T = list(product(G,T))

#-----------------
# Parámetros
#-----------------
d={# Periodo: demanda en kWh
   1: 100, 2: 150, 3: 150, 4: 200, 5: 100, 6: 100,
   7: 400, 8: 100, 9: 100, 10: 100, 11: 100, 12: 100} 
b={# Generadora: consumo maximo en kWh
   "Generadora 1":250,
   "Generadora 2":250,
   "Generadora 3":250}

m={# Generadora: duración de por contrato en meses
   "Generadora 1":2,
   "Generadora 2":4,
   "Generadora 3":3}

# Funciones de aproximación
# Función 1 evaluada en 20:  0.480 + 0.048 * (x - 20)
# Función 1 evaluada en 50:  3.000 + 0.120 * (x - 50)
# Función 1 evaluada en 100:  12.000 + 0.240 * (x - 100)
# Función 1 evaluada en 200:  48.000 + 0.480 * (x - 200)
# Función 2 evaluada en 20:  26.120 + -0.388 * (x - 20)
# Función 2 evaluada en 50:  17.000 + -0.220 * (x - 50)
# Función 2 evaluada en 100:  13.000 + 0.060 * (x - 100)
# Función 2 evaluada en 200:  47.000 + 0.620 * (x - 200)
# Función 3 evaluada en 20:  9.851 + -0.507 * (x - 20)
# Función 3 evaluada en 50:  2.535 + -0.049 * (x - 50)
# Función 3 evaluada en 100:  9.727 + 0.297 * (x - 100)
# Función 3 evaluada en 200:  58.768 + 0.644 * (x - 200)
f={#Generadora: lista de tuplas (a,p), donde "a" es el 
   #            intercepto y "p" la pendiente de una función
   #            aproximando el costo de la generadora
   "Generadora 1":[(0.48 - 0.048*20, 0.048),
                   (3.0 - 0.12*50, 0.12),
                   (12 - 0.24*100, 0.24),
                   (48 - 0.48*200, 0.48)],
   "Generadora 2":[(26.12 + 0.388*20, -0.388),
                   (17 + 0.22*50, -0.22),
                   (13 - 0.06*100, 0.06),
                   (47 - 0.62*200, 0.62)],
   "Generadora 3":[(9.851 + 0.507*20, -0.507),
                   (2.535 + 0.049*50, -0.049),
                   (9.727 - 0.297*100 , 0.297),
                   (58.768 - 0.644*200, 0.644)]}

#-------------------------------------
# Creación del objeto problema en PuLP
#-------------------------------------
#Crea el problema para cargarlo con la instancia 
problema=lp.LpProblem("CryptOT",lp.LpMinimize)

#-----------------------------
# Variables de Decisión
#-----------------------------
# Consumo mensual (en kWh) de la generadora i en G en el periodo
# de tiempo t en T
limite_superior_x = np.max([b[i] for i in G])
x=lp.LpVariable.dicts('x', G_x_T, lowBound=0, upBound=limite_superior_x,
                      cat=lp.const.LpContinuous)

# Variable binaria que indica si un nuevo contrato es firmado con la
# generadora i en G en el periodo de tiempo t en T
y=lp.LpVariable.dicts('y', G_x_T, cat=lp.const.LpBinary)

# Variable binaria que indica si hay un contrato vigente con la
# generadora i en G en el periodo de tiempo t en T
v=lp.LpVariable.dicts('v', G_x_T, cat=lp.const.LpBinary)


# Variable auxiliar para calcular el costo por consumo asociado
# a la generadora i en G en el periodo de tiempo t en T
z=lp.LpVariable.dicts('z', G_x_T, lowBound=0, cat=lp.const.LpContinuous)

#-----------------------------
# Función objetivo
#-----------------------------
#Crea la expresión de minimización de costos
problema+=lp.lpSum(z[i,t] for i in G for t in T), "Costo anual"

#-----------------------------
# Restricciones
#-----------------------------


# Satisfacción de la demanda
# sum(i in G)x_it = d_t forall t in T
for t in T:
    problema+= lp.lpSum(x[i,t] for i in G) == d[t], f"Satisfacción de la demanda en el mes {t}" 

# Relación entre consumo y contratos vigentes
# x_it <= b_i * v_it forall i in G, t in T
for (i,t) in G_x_T:
    problema+= x[i,t] <= b[i]*v[i,t], f"Consumo de la generadora {i} en el mes {t}" 

# Duracion de los contratos
for (i,t) in G_x_T:
    # Periodos en los que podría ser firmado un contrato para que este vigente en t.
    T_f = [tt for tt in range(t - m[i] + 1, t + 1) if tt>=1] 
    problema+= sum(y[i,tt] for tt in T_f) >= v[i,t], f"Vigencia en el mes {t} de contratos de la Generadora {i} parte 1"
    # Periodos vigentes si se firma un contrato en t
    T_v = [tt for tt in range(t, t + m[i]) if tt<=12] 
    problema+= len(T_v) * y[i,t] <= sum(v[i,tt] for tt in T_v), f"Vigencia en el mes {t} de contratos de la Generadora {i} parte 2"

# Máximo un contrato vigente por generadora en cada mes
# sum(tt in [t, ..., t+m_i-1])y_i,tt <= 1 forall i in G, t in T
for (i,t) in G_x_T:
    # Periodos vigentes si se firma un contrato en t
    T_v = [tt for tt in range(t, t + m[i]) if tt<=12] 
    problema+= lp.lpSum(y[i,tt] for tt in T_v) <= 1, f"Limite de nuevos contratos de la Generadora {i} firmados en el mes {t}" 

# Función aproximada a trozos
# z_it >= a_ik * p_ik * x_it - M(1-v_it)forall i in G, t in T, k = 1,...,numero_de_funciones
M = 100
for (i,t) in G_x_T:
    for k, (a,p) in enumerate(f[i]):
        problema+= z[i,t] >= a+ p*x[i,t] - M * (1-v[i,t]), f"Aproximación {k} del costo por consumo de la generadora {i} en el mes {t}" 

#-----------------------------
# Imprimir formato LP
#-----------------------------
#Escribe el problema en un archivo con formato LP 
problema.writeLP("CryptOR.lp")

#-----------------------------
# Invocar el optimizador
#-----------------------------
#Optimizar el modelo con CBC (default de PuLP)
problema.solve()

#-----------------------------
#    Imprimir resultados
#-----------------------------
#Imprimir estado final del optimizador
print("Estado (optimizador):", lp.LpStatus[problema.status],end='\n')

#Valor óptimo del plan de transporte  
print("\n Costos totales = $", round(lp.value(problema.objective),2))
print()

#Imprimir variables de decisión
print("Variables de decisión")
tap_ = ""
print(f'{"Mes":10s} {tap_.join(f"{t:>8d}" for t in T)}')
    
for i in G:
    print(f'{i:10s} {tap_.join(f"{x[i,t].value():>8.2f}" for t in T)}')
    print(f'{"Nuevo":10s} {tap_.join(f"{y[i,t].value():>8.0f}" for t in T)}')
    print(f'{"Vigente":10s} {tap_.join(f"{v[i,t].value():>8.0f}" for t in T)}')

## Créditos
Instancia: Daniel Duque<br>
Fecha: 21/12/2021